Band pass filtering

In [ ]:
def band_pass_filter(raw, l_freq=FMIN, h_freq=FMAX):
    raw.filter(method= 'fir',
        phase= 'minimum',
        fir_window= 'hann',
        l_freq= l_freq,
        h_freq= h_freq)
    return raw

Rereferencing

Calculates the mean voltage from all electrodes at each time point and subtracts this mean from the voltage at each individual electrode.

In [ ]:
def rereferencing(raw):
    raw.set_eeg_reference('average', projection=True).apply_proj() 
    return raw

NameError: name 'raw_cleaned' is not defined

Artifact Rejection (EOG/ECG) using Wavelet decomposition

In [ ]:
def wavelet_decompose(raw):
    info = raw.info
    coeffs = pywt.wavedec(raw.get_data(), 'db13', level=4)
    threshold = 0.00001 # applying a shrinkage function that smoothly brings coefficients below the threshold to zero
    coeffs_thresholded = [pywt.threshold(c, threshold, mode='soft') for c in coeffs] 
    denoised_signal = pywt.waverec(coeffs_thresholded, 'db13')
    raw = mne.io.RawArray(denoised_signal, info)
    return raw

Crop signal

In [ ]:
def crop(raw, l, h):
    return raw.crop(l,h)

Resampling

In [ ]:
def resampling(raw, SFREQ=SFREQ):
    return raw.resample(sfreq=SFREQ)

Drop channels

Dropping extra channels in both groups
TODO: Instead of dropping we can convert 10-10 64 channels montage to 10-20 32 channels

In [ ]:
def drop_channels(raw, raw_MDD):
    drop = []
    for chan in raw.info['ch_names']:
        if chan not in raw_MDD.info['ch_names']:
            drop.append(chan)
    raw.drop_channels(drop)
    return raw

MDD group preprocessing

In [ ]:
def MDD_preprocess(raw):
    raw = set_montage(raw)
    raw  = band_pass_filter(raw, l_freq = 0.01, h_freq = 45)
    raw = rereferencing(raw)
    # raw = wavelet_decompose(raw)
    raw.info['bads'] = []
    return raw

Control group preprcessing

In [ ]:
def control_preprocessing(raw, raw_MDD, crop=True):
    # For epochs crop=False
    raw = drop_channels(raw, raw_MDD)
    if crop:
        raw = set_montage(raw)
        raw = raw.crop(50, 200)
    raw = resampling(raw)
    raw = band_pass_filter(raw, l_freq = 0.01, h_freq = 45)
    # raw = wavelet_decompose(raw)
    raw.info['bads'] = []
    return raw